In [38]:
STREET_NAME = 'PARK AVE EXT'
STREET_NUM = '99'

%matplotlib widget
import  pandas            as pd
import  geopandas         as gpd
import  rasterio          as rs
import  rasterio.plot     as rsp
import  matplotlib.pyplot as plt


def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

## Download 209Mb zip file and store locally for repeated access
## https://documents.arlingtonma.s3.amazonaws.com/GIS_Data/ForDownload/ArlingtonMA_Ortho_2018_80r.zip?

data_dir   =  '/data/code/web/truepersons/data/public/US/MA/Arlington/'
arlington  =  rs . open ( data_dir + "Arlington_2018_80r.jp2" ) 

github_raw_url = 'https://gist.githubusercontent.com/StephenHarrington/c364e9125b87c703ae8307f9fe87795f/raw/'+\
                                                '51f76edd405b92ea2b3d173a40cb6f2b0fd8c5ea/ArlingtonMA.geojson'
df    =  gpd . read_file ( github_raw_url ) . to_crs ( arlington . crs ) 
df    =  unpack ( df , 'location' )
mask  =  ( df . street_number == STREET_NUM ) & ( df . street_name == STREET_NAME )

left , bottom , right , top = tuple ( map ( tuple , df [ mask ] . bounds . values ) ) [ 0 ]

fig , ( ax1 , ax2 )  =  plt . subplots ( 1 , 2 , figsize = ( 12 , 6 ) )

src  =  arlington . read ( window = rs . windows . from_bounds ( left , bottom , right , top ,  arlington . transform ) )
foo = rsp . show ( src , transform = arlington . transform , ax = ax1 )


rsp . show_hist ( src , bins = 50 , histtype = 'stepfilled' , lw = 0.0 , stacked = False , alpha = 0.3 , ax = ax2 )

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
type(foo)
df.iloc[0]

stroke-width                                                    2
fill-opacity                                                  0.5
fill                                                      #ffff00
tokenId                                       7345990835614514000
image           https://www.google.com/maps/@?api=1&map_action...
name                                                57 HIBBERT ST
description     A photovoltaic system was installed at 57 HIBB...
financials      {'contractor': ''CSG Services'', 'permits': '6...
dates                     {'effective_date_permit': '2003-08-06'}
location        {'coord': '(42.4237684, -71.1934237)', 'street...
ohm             {'parcel_size': 0.08, 'style': 'Colonial', 'ye...
governance      {'parcel': '176.0-0001-0009.0', 'land_use_code...
geometry2       POLYGON ((-71.1933435 42.4237918, -71.1933652 ...
attributes                                                   None
entity                                                       None
amps      

In [40]:
fig.savefig("/data/myplot.png")

In [ ]:
%matplotlib widget
import  json
import  pandas            as pd
import  geopandas         as gpd
import  rasterio          as rs
import  rasterio.plot     as rsp
import  matplotlib.pyplot as plt

fig , ax1  =  plt . subplots ( 1 , 1 , figsize = ( 12 , 6 ) )

## Download 209Mb zip file and store locally for repeated access
## https://documents.arlingtonma.s3.amazonaws.com/GIS_Data/ForDownload/ArlingtonMA_Ortho_2018_80r.zip?

data_dir      =   '/data/code/web/truepersons/data/public/US/MA/Arlington/'
solar_dir     =   '/data/code/web/truepersons/nft/solar/'
arlington     =   rs . open ( data_dir + "Arlington_2018_80r.jp2" ) 

df    =  gpd . read_file ( '/data/energy/REC/MA/Arlington/solar_systems.geojson' ) . to_crs ( arlington . crs ) 
df . insert ( loc = 4 , column =  'map' , value = df . image )
df [ 'image' ]    =  solar_dir + 'img/' + df . tokenId + ".png"

cols  =  df.columns

for idx in range ( len ( df ) ) :
    
    tokenId = df . iloc [ idx ] . tokenId
    left , bottom , right , top = df .iloc [ idx ] . geometry . bounds 
    
    with open ( solar_dir + 'metadata/' + tokenId + '.json' , "w" ) as outfile:
        outfile . write ( df [ cols [ : -1 ] ] . iloc [ idx ] . to_json ( ) )

    src  =  arlington . read ( window = rs . windows . from_bounds ( left , bottom , right , top ,  arlington . transform ) )
    rsp . show ( src , transform = arlington . transform , ax = ax1 )
    fig . savefig ( df . iloc [ idx ] . image )

    #plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
df . iloc [ idx ] . image
df[cols[0:-1]].iloc[idx]#.to_json()

tokenId                                      7345996461349260373
image          /data/code/web/truepersons/nft/solar/img/73459...
name                                                  1 CROSS ST
description    A 2.61kW photovoltaic system was installed at ...
map            https://www.google.com/maps/@?api=1&map_action...
attributes     {'expansion_system': 0, 'multiple_phase_system...
entity         {'applicant': 'Knollwood Energy, LLC', 'instal...
financials     {'applicant': 'Knollwood Energy, LLC', 'progra...
dates          {'effective_date': '2017-05-30', 'effective_da...
location       {'coord': '(42.4044766, -71.1351093)', 'street...
amps           {'utility': 'NSTAR (DBA EverSource)', 'utility...
watts          {'kW': 2.61, 'system_size_DC': 2.61, 'est_annu...
joules                                                        {}
ohm            {'parcel_size': 0.1, 'style': 'Old Style', 'ye...
xref           {'pts_index': 314, 'rps_index': 929, 'bnl_inde...
governance     {'parcel':

In [64]:
import fiona; help(fiona.open)

Help on function open in module fiona:

open(fp, mode='r', driver=None, schema=None, crs=None, encoding=None, layer=None, vfs=None, enabled_drivers=None, crs_wkt=None, **kwargs)
    Open a collection for read, append, or write
    
    In write mode, a driver name such as "ESRI Shapefile" or "GPX" (see
    OGR docs or ``ogr2ogr --help`` on the command line) and a schema
    mapping such as:
    
      {'geometry': 'Point',
       'properties': [('class', 'int'), ('label', 'str'),
                      ('value', 'float')]}
    
    must be provided. If a particular ordering of properties ("fields"
    in GIS parlance) in the written file is desired, a list of (key,
    value) pairs as above or an ordered dict is required. If no ordering
    is needed, a standard dict will suffice.
    
    A coordinate reference system for collections in write mode can be
    defined by the ``crs`` parameter. It takes Proj4 style mappings like
    
      {'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'W